In [57]:
import numpy as np
import matplotlib.pyplot as ptl 
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import pandas as pd
import struct

Recolección de data

In [98]:
def create_string_map(df_column):
    # Get all unique strings in the column
    unique_strings = df_column.unique()
    
    # Create a dictionary to map each unique string to an integer
    string_map = {string: i for i, string in enumerate(unique_strings)}
    
    return string_map

def apply_string_map(df_column, string_map):
    # Apply the string map to the column
    mapped_column = df_column.map(string_map)
    
    return mapped_column


* Ubigeo es el codigo de la localizacion del proyecto de tipo deparamento+provincia+ciudad concatenado
* Division_FN es la division del ministerio que se está encargando del proyecto
* Especifica_det es el tipo de especifico que se esta realizando
* Avanze_Fisico_sem es el avanze fisico del proyecto

In [102]:
train_df= pd.read_csv('data/train.csv', sep=',')
train_df['DIVISION_FN']=train_df['DIVISION_FN'].str.replace(r'\D+', '', regex=True)
train_df['ESPECIFICA_DET'] = apply_string_map(train_df['ESPECIFICA_DET'], create_string_map(train_df['ESPECIFICA_DET']))
selection = ['UBIGEO', 'DIVISION_FN', 'ESPECIFICA_DET', 'AVAN_FISICO_SEM']
train_df[selection][1:60]


,UBIGEO,DIVISION_FN,ESPECIFICA_DET,AVAN_FISICO_SEM
1,230199,006,1,6
2,10199,051,2,16
3,150101,006,3,6
4,50699,006,4,6
5,170199,051,5,2
6,10199,051,6,16
7,80199,051,5,63
8,90199,006,7,6
9,40199,051,8,17
10,40199,006,9,6


In [4]:
class LinearRegresion:
    
    def __init__(self,dim):
        self.dim = dim
        self.m_W = np.random.rand(dim)
        self.m_b = np.random.random()
        
    def H(self,X):
        return np.dot(X,self.m_W) + self.m_b

    
    def Loss(self,X,Y):
        y_pred = self.H(X)
        return (np.linalg.norm((Y   - y_pred))**2)/(2*len(Y)), y_pred

    def dL(self,X,Y, Y_pre):
        dw =  np.matmul(Y - Y_pre,-X)/len(Y)
        db =  np.sum((Y - Y_pre)*(-1))/len(Y)
        return dw,db 

    def change_params(self, dw,db, alpha):
        self.m_W = self.m_W - alpha*dw 
        self.m_b = self.m_b - alpha*db

    def train(self, X, Y, alpha,epochs):
        error_list = []
        time_stamp = []
        for i in range(epochs):
            loss, y_pred = self.Loss(X,Y)
            time_stamp.append(i)
            error_list.append(loss)
            dw,db = self.dL(X,Y,y_pred)
            self.change_params(dw,db,alpha)
            # print("error de pérdida : " + str(loss))
            if(i%100==0):
            #    self.plot_error(time_stamp, error_list)
                 self.plot_plane(X[:,0],X[:,1],Y)
            
        return time_stamp, error_list
       

    def plot_error(self, time, loss):
       ptl.plot(time, loss)
       ptl.show()
  
    def plot_line(self,x,y):
       ptl.plot(x, y,'*')
       ptl.plot(x,self.H(x))
       ptl.show()
       
    def plot_plane(self,xx,yy,zz):
        fig = ptl.figure(figsize=(8, 8))
       
        
        # crear un subplot 3D
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(xx, yy, zz, c='r', marker='o')
        
        # definir el tamaño del plano
        x = np.linspace(min(xx), max(xx),len(xx))  
        y = np.linspace(min(yy), max(yy),len(yy))
        
  
        x, y = np.meshgrid(x, y)
        
        # definir la ecuación del plano
        z = self.m_W[0]*x + self.m_W[1]*y+ self.m_b 
      
        
        # dibujar el plano
        ax.plot_surface(x, y, z )
        ax.plot_surface

        ax.view_init(elev=0, azim=70) 
        
        # agregar etiquetas a los ejes
        ax.set_xlabel('Feacture 1: X0')
        ax.set_ylabel('Feacture 2: X1')
        ax.set_zlabel('Prediction')
        
       
        ax.mouse_init()

        ptl.show()  